<a href="https://colab.research.google.com/github/afikbar/machine_learning/blob/master/Learning_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import glob
import scipy.misc
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import imageio
import skimage
from sklearn.naive_bayes import MultinomialNB
import os

In [0]:
path = "/content/drive/My Drive/Technion/Semester 6/Machine Learning/Final Project"

data = pd.read_csv(path + "/MovieGenre.csv", encoding="ISO-8859-1")
data = data.dropna(subset=['Genre', 'Poster'])

In [4]:
idx, row = next(data.iterrows())
row.Genre

'Animation|Adventure|Comedy'

In [0]:
def prepare_data(data, size=(268, 182)):
    print("Generation dataset...")
    dataset = []
    y = []
    ids = []
    label_dict = {"word2idx": {}, "idx2word": []}
    idx = 0
    genre_per_movie = data["Genre"].apply(lambda x: str(x).split("|"))
    for l in [g for d in genre_per_movie for g in d]:
        if l in label_dict["idx2word"]:
            pass
        else:
            label_dict["idx2word"].append(l)
            label_dict["word2idx"][l] = idx
            idx += 1
    n_classes = len(label_dict["idx2word"])
    print("identified {} classes".format(n_classes))
    for idx, row in data.iterrows():
      splitted_genres = row.Genre.split('|')
      image_path = path + "/posters/{}.jpg".format(row.imdbId)
      if os.path.exists(image_path):
        try:
          img = imageio.imread(image_path)
          if img.shape != (268, 182, 3):
              continue
          img = img.reshape(146328)
          l = np.sum([np.eye(n_classes, dtype="uint8")[label_dict["word2idx"][s]] 
                                                      for s in splitted_genres], axis=0)
          y.append(l)
          dataset.append(img)
          ids.append(row.imdbId)
        except FileNotFoundError as e:
          pass
        except Exception as e:
          print("Error in: ",row.imdbId, "\n" , e)

    print("DONE")
    return dataset, np.array(y), label_dict, ids

In [0]:
dataset, y, label_dict, ids =  prepare_data(data)

Generation dataset...
identified 28 classes
Error in:  112541 
 'NoneType' object has no attribute 'ReadAsArray'
Error in:  113044 
 'NoneType' object has no attribute 'ReadAsArray'


In [0]:
NB = MultinomialNB()
NB.fit(dataset[:10000], y[:10000,6])